In [1]:
#Wenn man den typen bestimmte hat, sollte man zunächst alle Pflicht-Felder des Bibtex-Eintrages raussuchen und bestimmen.
#Dazu die Pflichtattribute in einer Schleife durchgehen.

In [2]:
import torch
import pandas as pd
torch.cuda.is_available()
from transformers import pipeline
import re
import string

@book{advanced_physics, author = {Stephen Hawking and Brian Greene and Maria Trump and Susan Soy}, title = {Advanced Concepts in Theoretical Physics}, publisher = {Cambridge University Press}, year = {2025}, edition = {3}, volume = {1}, series = {Advanced Studies in Physics}, address = {Cambridge, UK}, month = {May}, note = {A seminal work in the field of theoretical physics}, isbn = {978-0-521-76948-0}, doi = {10.1017/example.book.2025}, url = {https://www.cambridge.org/advanced_physics}, annote = {Widely cited in the physics community}, abstract = {This book explores cutting-edge theories and concepts in theoretical physics...}, keywords = {Theoretical Physics, Quantum Mechanics, String Theory}, language = {English}, price = {75.00}, size = {600 pages}, lccn = {2020934576}, mrnumber = {MR3070071} }

@article{quantum_entanglement,
  author        = {Albert Einstein and Boris Podolsky and Nathan Rosen},
  title         = {Can Quantum-Mechanical Description of Physical Reality Be Considered Complete?},
  journal       = {Physical Review},
  year          = {1935},
  volume        = {47},
  number        = {10},
  pages         = {777-780},
  month         = {May},
  note          = {EPR Paradox paper, fundamental for quantum mechanics},
  doi           = {10.1103/PhysRev.47.777},
  url           = {https://journals.aps.org/pr/abstract/10.1103/PhysRev.47.777},
  abstract      = {In this paper, the authors discuss the EPR paradox and challenge the completeness of quantum mechanics...},
  keywords      = {Quantum Mechanics, EPR Paradox, Physical Reality},
  language      = {English},
  publisher     = {American Physical Society}
}

@inproceedings{deep_learning,
  author        = {Geoffrey Hinton and Yoshua Bengio and Yann LeCun},
  title         = {Deep Learning for Artificial Intelligence},
  booktitle     = {Proceedings of the IEEE International Conference on Neural Networks},
  year          = {2021},
  editor        = {Jane Smith and John Doe},
  volume        = {1},
  number = {5},
  series        = {Advances in Neural Information Processing},
  pages         = {100-120},
  address       = {Montreal, Canada},
  month         = {December},
  organization  = {IEEE},
  publisher     = {IEEE Press},
  note          = {Keynote paper on recent advancements in deep learning},
  isbn          = {978-1-5386-4637-1},
  doi           = {10.1109/ICNN.2021.9483948},
  url           = {https://ieeexplore.ieee.org/document/9483948},
  annote        = {A seminal work on how deep learning transforms AI},
  abstract      = {This paper explores cutting-edge deep learning techniques and their impact on the development of artificial intelligence...},
  keywords      = {Deep Learning, Artificial Intelligence, Neural Networks},
  language      = {English}
}


@incollection{quantum_computation,
  author        = {Michael A. Nielsen and Isaac L. Chuang},
  title         = {Quantum Computation and Quantum Information},
  booktitle     = {Handbook of Quantum Information Science},
  publisher     = {Springer},
  year          = {2026},
  editor        = {Charles H. Bennett and David P. DiVincenzo},
  volume        = {4},
  series        = {Quantum Science and Technology},
  chapter       = {10},
  pages         = {250-300},
  address       = {Berlin, Germany},
  month         = {October},
  note          = {A comprehensive overview of the fundamentals of quantum computation},
  isbn          = {978-3-540-88702-7},
  doi           = {10.1007/springerreference_303198},
  url           = {https://www.springer.com/gp/book/9783540887027},
  annote        = {Essential reading for researchers entering the field of quantum information},
  abstract      = {This chapter delves into the principles of quantum computing, offering an accessible yet thorough introduction...},
  keywords      = {Quantum Computing, Quantum Information, Computational Models},
  language      = {English},
  price         = {45.00},
  size          = {50 pages}
}


In [3]:
#RegEx zum FInden der Seiten
pageFinder = ", (?:pp\.? )?\d+-\d+,"
doi1 = "https:\/\/doi\.org"
doi2 = "DOI:"

#if havard und book:
volume = ", \d+,"
number = ", \d+,"
edition1 = "(?:[1-9]\d*th|11th|12th|13th|[1-9]\d*(?:st|nd|rd)) ed\."
edition2 = "(?:[1-9]\d*th|11th|12th|13th|[1-9]\d*(?:st|nd|rd)) edn\."

#if APA or Havard then
volume = "\(Vol\. \d+" #für Volume und number noch als Bedingung, dass es nach Titel stehen muss, also Index > Titel-Index
number = "Issue \d+"
yearApaHavard = "\(\d{4}\)"
editorRegEx = " (\()?(Eds\.|Eds|Ed|ed|Ed\.|ed\.|eds\.|editor|editors|edited by )(\))?"

#editor bei anderen


#Beachte: Editor und Edition werden oft mit gleichen Abkürzungen versehen, auch im gleichen Stil! Nur Edition wird oft
#klein geschrieben. Daher zustzlich prüfen, ob NER das als Person erkennt.

#Beachte: ieee und mla trennen mit , und nicht mit .
#if ieee and mla
volumeIeeeMla = ", vol\. \d+" 
noIeeeMla = ", no\. \d+"

etAl = "et al."

yearACM = "\. \d{4}\."

#Man sollte zunächst die Felder extrahieren, die zu 100% sicher erkennen kann wie Volume oder Number, doi, Autoren...-> 
#Titel, booktitle, series und journal danach

#man sollte eine Überdeckungsprüfung machen: Also Bspw. sagt Space im Bereich 25-50 ist eine Orga, Regex sagt, 
#dort ist ein (Eds.)
#zu finden und Huggingface sagt, von 25-45 ist was. Dann sollte der minimalteste Wert und maximalste Wert 
#genommen werden und so lange nach links und analog nach rechts gehen, bis wieder ein Punkt kommt.
# Da erkannte Felder aus dem String entfernt werden, muss der String am Ende leer sein. 
#Daher sollten zunächste die Dinge ausgeschnitten werden wo der Algo sich am sichersten ist
#TODO: Grundsätzliche Reihenfolge der BibTex-Einträge ermitteln.
#TODO: Für jede Extrahierung eines Feldes eine eigene Unterroutine schreiben? -> In der Oberschleife sollte Style sein



print(re.search(editorRegEx, "Singh, S. K., Kumar, S., & Mehra, P. S. (2023). Chapter Title. In Johnson, E. F. & Lee, R. H. (Eds.), Book"))

<re.Match object; span=(93, 100), match=' (Eds.)'>


In [35]:
text="Hawking, S., Greene, B., Trump, M. and Soy, S. (2025). Advanced Concepts and John Miller in Theoretical Physics (3rd ed., Vol. 1). Cambridge University Press. https://doi.org/10.1017/example.book.2025"
text = "Singh, S. K., Kumar, S., & Mehra, P. S. (2023). Chat GPT & Google Bard AI: A Review. 2023 International Conference on IoT, Communication and Automation Technology (ICICAT), 1–6. https://doi.org/10.1109/ICICAT57735.2023.10263706"
text = "Singh, S. K., Kumar, S., & Mehra, P. S. (2023). An new model for nuclear Energy. In Johnson, E. F. & Lee, R. H. (Eds.), Nuclear Energy Today. Publisher Name."
#text = "Singh, S. K. and Peterman, P. (2023). Chapter and Title. Trump, M. and Soy, S. Book Title. Publisher Name."
#text = "S. K. Singh, S. Kumar & P. S. Mehra, (2023). in Wonderland. In F. Johnson & L. Lee (Eds.), Book Title. Publisher Name."
#text = "F Johnson & L Lee (Eds.)"
#text = "Tu Anh Nguyen, Eugene Kharitonov, Jade Copet, Yossi Adi, Wei-Ning Hsu, Ali Elkahky, Paden Tomasello, Robin Algayres, Benoît Sagot, Abdelrahman Mohamed, and Emmanuel Dupoux. 2023. Generative spoken dialogue language modeling. Transactions of the Association for Computational Linguistics 11, (2023), 250–266. URL: https://aclanthology.org/2023.tacl-1.15, doi:10.1162/tacl_a_00545"
#text = "Hinton G, Bengio Y, LeCun Y (2021) Deep Learning for Artificial Intelligence. In: Smith J, Doe J (eds) Proceedings of the IEEE International Conference on Neural Networks. IEEE Press, Montreal, Canada, pp 100–120"


In [43]:
text = "Singh, S. K., Kumar, S., & Mehra, P. S. (2023). An new model for nuclear Energy. In Johnson, E. F. & Lee, R. H. (Eds.), Nuclear Energy Today. Publisher Name."


def find_First_Term(text, search_terms):
    # Initialisiere mit einem hohen Wert
    min_index = float('inf')
    end_index = 0
    andTyp = ""
    
    # Suche jeden Suchbegriff in dem Text und behalte den kleinsten Index
    for term in search_terms:
        index = text.find(term)
        if index != -1 and index < min_index:
            min_index = index
            end_index = min_index + len(term) - 1
            andTyp = term
    
    # Wenn min_index unverändert ist, wurde keiner der Begriffe gefunden
    return (min_index, end_index, andTyp) if min_index != float('inf') else (-1, -1, "")

def is_SurenameFirst(names):
    if re.search(r'^(\w+\.)', names) or re.search(r'^(\w+\s\w+(.,)?)+$', names):
        return True
    return False
    
def is_NameShortened(df_PER):
    for index in df_PER.index.values.tolist():
        if "." == text[df_PER["end"].iloc[index]] and len(text[df_PER["start"].iloc[index]:df_PER["end"].iloc[index] + 1]) == 2:
            return True
    return False

def getIndexOfSubstring(regEx, text):
    if re.search(regEx, text):
        x = re.search(editorRegEx, text)
        return (x.start(), x.end())
    else:
        return(-1,-1)

def is_punctuation(s):
    allowed_chars = string.punctuation.replace('&', '') + ' '
    return all(char in allowed_chars for char in s)

def is_Editor(editorRegEx, textBetweenNames, index):
    if re.search(editorRegEx, textBetweenNames):
        x = re.search(editorRegEx, text)
        #print(f'x: {x.start()}')
        if is_punctuation(text[index:x.start()]):
            return True
    return False

def extract_Names(authors):
    finalAuthors = ""
    search_terms = [" and ", ", and ", " & ", ", & "]
    andInAuthors = find_First_Term(authors, search_terms)[2]
    surenameFirst = is_SurenameFirst(authors)
    print(f'surenameFirst: {surenameFirst}')
    if surenameFirst:
        #hier völlig egal, ob er einzelne Initialen in ein eigenes Word gesteckt hat, obwohl es noch Nachnamen gib
        authors = authors.replace(andInAuthors, " and ")
        print(f'authors: {authors}')
        finalAuthors = authors.replace(", ", " and ")
    elif andInAuthors != "":
        if "., " in authors:
            print("Fall ., {0}".format(authors))
            search_terms = ["., and ", "., & ", ". and ", ". & "]
            andInAuthors = find_First_Term(authors, search_terms)[2]
            authors = authors.replace(andInAuthors, "., ")
            authors = authors.split("., ")
            authors = [name + "." for name in authors]
            authors = [name.replace("..",".") for name in authors]
            for author in authors[:-1]:
                buffer = author.split(", ")
                finalAuthors = finalAuthors + buffer[1] + " " + buffer[0] + " and "
            buffer = authors[-1].split(", ")
            finalAuthors = finalAuthors + buffer[1] + " " +  buffer[0]
        elif ", " in authors:
            print("Fall , {0}".format(authors))
            search_terms = [", and ", ", & ", " and ", " & "]
            andInAuthors = find_First_Term(authors, search_terms)[2]
            authors = authors.replace(andInAuthors, ", ")
            authors = authors.split(", ")
            for i in range(0, len(authors) - 3, 2):
                finalAuthors = finalAuthors + authors[i+1] + " " + authors[i] + " and "
            finalAuthors = finalAuthors + authors[len(authors) - 1] + " " + authors[len(authors) - 2]
    else:
        print("Fall else {0}".format(authors))
        authors = authors.split(", ")
        finalAuthros = authors[1] + authors[0]
    return finalAuthors

def getAuthorsAndEditors(df_PER, text):
    search_terms = [" and ", ", and ", " & ", ", & ", "., & ", "., and ", ". and ", ". & "]
    editorRegEx = " (\()?(Eds\.|Eds|Ed|ed|Ed\.|ed\.|eds\.|editor|editors)(\))?"
    index_df_PER_List = df_PER.index.values.tolist()
    onlyPunctuation = False
    onlyAnd = False
    authorsDetected = False
    setChainStart = True
    startIndexAuthors = -1
    endIndexAuthors = -1
    startIndexEditors = -1
    endIndexEditors = -1
    chainStartIndex = -1


    for index in index_df_PER_List:
        #beachte: Hiermit lese ich immer schon vor!
        if index < len(index_df_PER_List) - 1:
            textBetweenNames = text[df_PER["end"].iloc[index]:df_PER["start"].iloc[index + 1]]
        else:
            textBetweenNames = text[df_PER["end"].iloc[index]:]
        onlyPunctuation = is_punctuation(textBetweenNames)
        print(f'getAuthorsAndEditors: textBetweenNames: {textBetweenNames}')
        print(f'getAuthorsAndEditors: Author: {text[df_PER["start"].iloc[index]:df_PER["end"].iloc[index]]}')
        print(f'getAuthorsAndEditors: onlyPunctuation: {onlyPunctuation}')
        firstStartIndex, firstEndIndex, andTyp = find_First_Term(textBetweenNames, search_terms)
        onlyAnd = textBetweenNames == andTyp
        print(f'getAuthorsAndEditors: onlyAnd: {onlyAnd}')
        print(f'getAuthorsAndEditors: nothing: {not onlyAnd and not onlyPunctuation}')
        if setChainStart: 
            chainStartIndex = df_PER["start"].iloc[index]
            #Solange das auf False, sollen der Substring erweitert werden, also start bleibt konstant
            #print(f'chainStartIndex: {chainStartIndex}')
            setChainStart = False
        #Dann gab es einen Bruch in der Autorenkette. Also bin ich in einer Lücke zwischen den AUtoren
        #Dann ist Nächster Block wieder ein Autor
        if not onlyPunctuation and not onlyAnd:
            setChainStart = True
            #Es können auch nur Editoren und keine Autoren vorkommen
            authorsDetected = not is_Editor(editorRegEx, textBetweenNames, df_PER["end"].iloc[index])
            print(f'getAuthorsAndEditors: authorsDetected: {authorsDetected}')
            print(f'getAuthorsAndEditors: is_Editor: {is_Editor(editorRegEx, textBetweenNames, df_PER["end"].iloc[index])}')
            print(f'getAuthorsAndEditors: chainStartIndex: {chainStartIndex}')
            # endIndexAuthors == -1, damit Autoren im Titel nicht wieder als Autoren erkannt werden
            if authorsDetected and endIndexAuthors == -1:
                startIndexAuthors = chainStartIndex
                endIndexAuthors = df_PER["end"].iloc[index]
            #nicht nur ein else, falls Namen im Titel des Buches auftauchen
            elif is_Editor(editorRegEx, textBetweenNames, df_PER["end"].iloc[index]):
                startIndexEditors = chainStartIndex
                endIndexEditors = df_PER["end"].iloc[index]
                break
    print(f'getAuthorsAndEditors: return: {[startIndexAuthors,endIndexAuthors],[startIndexEditors, endIndexEditors]}')
    return [[startIndexAuthors,endIndexAuthors],[startIndexEditors, endIndexEditors]]

def replaceSubstring (startIndex, endIndex, text, substituteString):
    startIndexReplace = -1
    endIndexReplace = -1
    if startIndex > 0:
        for i in range(startIndex - 1, 0, -1):
            if not is_punctuation(text[i]):
                startIndexReplace = i + 1
                break
    else:
        startIndexReplace = 0
        substituteString = ''
    for i in range(endIndex, len(text) - 1, 1):
        if not is_punctuation(text[i]):
            endIndexReplace = i
            break
    text = text[0:startIndexReplace] + substituteString + text[endIndexReplace:len(text) - 1]
    return text


#search_terms = [", et al.", " et al."]
#firstStartIndex, firstEndIndex, etAl = find_First_Term(text, search_terms)
#if firstStartIndex > -1:
    #text = replaceSubstring(firstStartIndex, firstEndIndex, text, ", ")


ner_tagger = pipeline("ner", aggregation_strategy="simple")
print(text)
outputs = ner_tagger(text)
df_outputs = pd.DataFrame(outputs)
print(df_outputs)
#index neu setzen, da diese nicht automatich geupdates werden
df_PER = df_outputs[df_outputs["entity_group"] == "PER"].reset_index(drop=True)


print(df_PER)

finalAuthors = ""
finalEditors = ""
auhtorsAndEditorsIndices = getAuthorsAndEditors(df_PER, text)
if auhtorsAndEditorsIndices[0][0] > -1:
    authors = text[auhtorsAndEditorsIndices[0][0]:auhtorsAndEditorsIndices[0][1]]
    print(f'authors : {authors}')
    finalAuthors = extract_Names(authors)
    #text = replaceSubstring(auhtorsAndEditorsIndices[0][0], auhtorsAndEditorsIndices[0][1], text, "")


if auhtorsAndEditorsIndices[1][0] > -1:
    editors = text[auhtorsAndEditorsIndices[1][0]:auhtorsAndEditorsIndices[1][1]]
    print(f'editors : {editors}')
    finalEditors = extract_Names(editors)
    text = replaceSubstring(auhtorsAndEditorsIndices[1][0], auhtorsAndEditorsIndices[1][1], text, ", ")
    indicesEditorMarker = getIndexOfSubstring(editorRegEx, text)
    #text = replaceSubstring(indicesEditorMarker[0], indicesEditorMarker[1], text, ", ")

print(text)
print(f'finalAuthors: {finalAuthors}')
print(f'finalEditors: {finalEditors}')

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
C:\Users\larsl\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTrai

Singh, S. K., Kumar, S., & Mehra, P. S. (2023). An new model for nuclear Energy. In Johnson, E. F. & Lee, R. H. (Eds.), Nuclear Energy Today. Publisher Name.
   entity_group     score                  word  start  end
0           PER  0.999318                 Singh      0    5
1           PER  0.996027                     S      7    8
2           PER  0.997318                     K     10   11
3           PER  0.999508                 Kumar     14   19
4           PER  0.997833                     S     21   22
5           PER  0.989305                 Mehra     27   32
6           PER  0.998831                     P     34   35
7           PER  0.997961                     S     37   38
8          MISC  0.566246                Energy     73   79
9           PER  0.999361               Johnson     84   91
10          PER  0.992642                     E     93   94
11          PER  0.985034                     F     96   97
12          PER  0.998692                   Lee    101  104
13

In [35]:
        firstStartIndex, firstEndIndex, editorTyp = find_First_Term(text, [" edited by "])
        print(f'firstEndIndex edited by: {firstEndIndex}')
        if firstStartIndex > -1 and index < len(index_df_PER_List) - 1:
            print(f'df_PER["start"].iloc[index]: {df_PER["start"].iloc[index + 1]}')
            editedBy = firstEndIndex + 1 == df_PER["start"].iloc[index + 1]
            #hier kam es oft vor, dass NER die Namen nicht richtig erkennt. Daher Sonderprüfung
            if not editedBy:
                buffer = text[firstEndIndex + 1:df_PER["start"].iloc[index + 1]]
                if re.search(r'^[A-Za-z]\.([ ][A-Za-z]\.)*[ ]?$', buffer):
                    editedBy = True
        print(f'editedBy: {editedBy}')
        if editedBy:
            startIndexEditorMarker = firstStartIndex
            endIndexEditorMarker = firstEndIndex
            editorsDetected = True
        else:
            print(f'x: {re.search(editorRegEx, text)}')
            
            
  
    return False

firstEndIndex edited by: 68
editedBy: False
x: <re.Match object; span=(59, 62), match=' ed'>


Idee: Nun zunächst Jahr, Volume, Seiten, Edition und URL/DOI extrahieren. Den Rest (also Titel, Publisher, Series), dann nochmal den SpacyParer drüber laufen lassen, weil 

In [16]:
s = "This Book was edited by Martin Trump" 
print(find_First_Term(s, ["edited by"]))
ner_tagger = pipeline("ner", aggregation_strategy="simple")
text = s
outputs = ner_tagger(text)
df_outputs = pd.DataFrame(outputs)
print(df_outputs)

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


(14, 22, 'edited by')


Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


  entity_group     score          word  start  end
0         MISC  0.871492          Book      5    9
1          PER  0.999499  Martin Trump     24   36


In [ ]:
#Test, um Datum Zuverlässig erkannt wird

import spacy
nlp = spacy.load("en_core_web_sm")
doc = nlp("Hinton, G., Bengio, Y. and LeCun, Y. (2021). Deep Learning for Artificial Intelligence. In J. Smith & J. Doe (Eds.), Proceedings of the IEEE International Conference on Neural Networks (Vol. 1, Issue 5, pp. 100–120). IEEE Press. https://doi.org/10.1109/ICNN.2021.9483948")
for ent in doc.ents:
  print(ent, ent.start_char-ent.sent.start_char, ent.end_char-ent.sent.start_char, ent.label_)

In [23]:

text = "Hinton, G., Bengio, Y. and LeCun, Y. (2021). Deep Learning for Artificial Intelligence. In J. Smith & J. Doe (Eds.), Proceedings of the IEEE International Conference on Neural Networks (Vol. 1, Issue 5, pp. 100–120). IEEE Press. https://doi.org/10.1109/ICNN.2021.9483948"
startIndex = 0
endIndex = 18
startIndexReplace = -1
endIndexReplace = -1
if startIndex > 0:
    for i in range(startIndex - 1, 0, -1):
        if not is_punctuation(text[i]):
            startIndexReplace = i + 1
            break
else:
    startIndexReplace = 0
for i in range(endIndex, len(text) - 1, 1):
    if not is_punctuation(text[i]):
        endIndexReplace = i
        break
text = text[0:startIndexReplace] + "#" + text[endIndexReplace:len(text) - 1]
print (text)

#Y. and LeCun, Y. (2021). Deep Learning for Artificial Intelligence. In J. Smith & J. Doe (Eds.), Proceedings of the IEEE International Conference on Neural Networks (Vol. 1, Issue 5, pp. 100–120). IEEE Press. https://doi.org/10.1109/ICNN.2021.948394
